In [1]:
    # hsv_color_dataset.py
    import os
    import numpy as np
    from PIL import Image
    import colorsys
    import math
    import random

    base_dir = "color_dataset_rgb_hsv"
    os.makedirs(base_dir, exist_ok=True)

    # base hues in degrees (0=red, 120=green, 240=blue)
    families = {
        "red": 0,
        "green": 120,
        "blue": 240
    }

    samples_per_family = 1000  # total images per family
    img_size = (64, 64)  # image dimensions

    def hsv_to_rgb_uint8(h_deg, s, v):
        # colorsys uses h in [0,1]
        h = (h_deg % 360) / 360.0
        r, g, b = colorsys.hsv_to_rgb(h, s, v)
        return (int(round(r * 255)), int(round(g * 255)), int(round(b * 255)))

    for family, base_h in families.items():
        folder = os.path.join(base_dir, family)
        os.makedirs(folder, exist_ok=True)
        for i in range(samples_per_family):
            # small hue jitter so we stay within the family
            #  -> +/- 8 degrees works well; adjust if needed
            h_jitter = random.uniform(-4, 4)
            s_values = np.linspace(0.4, 1.0, num=5)  # skip very low saturation
            v_values = np.linspace(0.2, 1.0, num=5)  # skip too dark

            # saturation and value jitter kept reasonably high to keep color identity
            s = random.choice(s_values)
            v = random.choice(v_values)

            rgb = hsv_to_rgb_uint8(base_h + h_jitter, s, v)

            # Create image with that color but add tiny per-pixel noise only in +/- 8 range
            base = np.ones((img_size[0], img_size[1], 3), dtype=np.int16) * np.array(rgb, dtype=np.int16)
            noise = np.random.randint(-6, 7, (img_size[0], img_size[1], 3), dtype=np.int16)  # small noise
            img_arr = np.clip(base + noise, 0, 255).astype(np.uint8)

            Image.fromarray(img_arr).save(os.path.join(folder, f"{family}_{i}.png"))

    print("✅ HSV-based RGB dataset created at:", base_dir)


✅ HSV-based RGB dataset created at: color_dataset_rgb_hsv


In [ ]:
# generate_varied_color_dataset.py
import os
import numpy as np
from PIL import Image, ImageDraw, ImageFilter
import colorsys
import random

base_dir = "color_dataset_varied"
os.makedirs(base_dir, exist_ok=True)

families = {
    "red": 0, 
    "orange": 30,   # hue degrees
    "yellow": 60,
    "green": 120,
    "blue": 240
    "indigo": 275,
    "violet": 300,
    "black": 0,
}

img_size = (64, 64)       # bigger -> more realistic patches
samples_per_family = 1000  # increase for training

def hsv_to_rgb_uint8(h_deg, s, v):
    h = (h_deg % 360) / 360.0
    r, g, b = colorsys.hsv_to_rgb(h, s, v)
    return (int(round(r * 255)), int(round(g * 255)), int(round(b * 255)))

def random_gradient(hue_deg, s, v):
    # create linear gradient with small hue & sat variation along one axis
    base = np.zeros((img_size[1], img_size[0], 3), dtype=np.uint8)
    for x in range(img_size[0]):
        t = x / (img_size[0] - 1)
        # vary value and saturation slightly across gradient
        s_x = np.clip(s * (1 - 0.2 * (0.5 - t)), 0, 1)
        v_x = np.clip(v * (0.6 + 0.4 * t), 0, 1)
        rgb = hsv_to_rgb_uint8(hue_deg + random.uniform(-4, 4), s_x, v_x)
        base[:, x, :] = rgb
    # add subtle per-pixel noise
    noise = np.random.randint(-8, 9, base.shape, dtype=np.int16)
    img = np.clip(base.astype(np.int16) + noise, 0, 255).astype(np.uint8)
    return img

def random_stripes(hue_deg, s, v):
    base = np.zeros((img_size[1], img_size[0], 3), dtype=np.uint8)
    n_stripes = random.randint(2, 6)
    stripe_width = img_size[0] // n_stripes
    for i in range(n_stripes):
        rgb = hsv_to_rgb_uint8(hue_deg + random.uniform(-6, 6),
                               np.clip(s * random.uniform(0.7, 1.0), 0, 1),
                               np.clip(v * random.uniform(0.6, 1.0), 0, 1))
        x0 = i * stripe_width
        x1 = img_size[0] if i == n_stripes - 1 else x0 + stripe_width
        base[:, x0:x1, :] = rgb
    noise = np.random.randint(-10, 11, base.shape, dtype=np.int16)
    img = np.clip(base.astype(np.int16) + noise, 0, 255).astype(np.uint8)
    return img

def random_blobs(hue_deg, s, v):
    # use PIL to draw blobs of slightly different hue/sat/value
    img = Image.new('RGB', img_size, hsv_to_rgb_uint8(hue_deg + random.uniform(-3,3), s, v))
    draw = ImageDraw.Draw(img)
    num_blobs = random.randint(3, 10)
    for _ in range(num_blobs):
        rx = random.randint(5, img_size[0]//2)
        ry = random.randint(5, img_size[1]//2)
        x = random.randint(0, img_size[0]-rx)
        y = random.randint(0, img_size[1]-ry)
        hue_j = hue_deg + random.uniform(-8, 8)
        rgb = hsv_to_rgb_uint8(hue_j,
                               np.clip(s * random.uniform(0.6, 1.0), 0, 1),
                               np.clip(v * random.uniform(0.5, 1.0), 0, 1))
        draw.ellipse([x, y, x+rx, y+ry], fill=rgb)
    # optionally blur slightly to get soft transitions
    img = img.filter(ImageFilter.GaussianBlur(radius=random.uniform(0.5, 1.8)))
    arr = np.array(img)
    noise = np.random.randint(-7, 8, arr.shape, dtype=np.int16)
    img_arr = np.clip(arr.astype(np.int16) + noise, 0, 255).astype(np.uint8)
    return img_arr

def random_texture(hue_deg, s, v):
    # Perlin-like simple texture: layered noise and a dominant hue tint
    base = np.zeros((img_size[1], img_size[0], 3), dtype=np.uint8)
    for _ in range(4):
        rgb = hsv_to_rgb_uint8(hue_deg + random.uniform(-5,5),
                               np.clip(s * random.uniform(0.6,1.0),0,1),
                               np.clip(v * random.uniform(0.5,1.0),0,1))
        layer = np.random.randint(0, 255, (img_size[1], img_size[0], 1), dtype=np.uint8)
        layer = np.repeat(layer, 3, axis=2)
        base = np.clip((base.astype(np.int16) + (layer.astype(np.int16) * np.array(rgb) // 255)), 0, 255).astype(np.uint8)
    noise = np.random.randint(-20, 21, base.shape, dtype=np.int16)
    img = np.clip(base.astype(np.int16) + noise, 0, 255).astype(np.uint8)
    return img

generators = [random_gradient, random_stripes, random_blobs, random_texture]

for family, base_h in families.items():
    folder = os.path.join(base_dir, family)
    os.makedirs(folder, exist_ok=True)
    for i in range(samples_per_family):
        hue_jitter = random.uniform(-6, 6)
        s = random.uniform(0.6, 1.0)
        v = random.uniform(0.5, 1.0)
        gen = random.choice(generators)
        img = gen(base_h + hue_jitter, s, v)
        # convert to uint8 just in case and save
        Image.fromarray(img.astype(np.uint8)).save(os.path.join(folder, f"{family}_{i}.png"))

print("✅ Varied dataset created at:", base_dir)


SyntaxError: invalid syntax. Perhaps you forgot a comma? (1457931754.py, line 16)

: 